**Copyright 2021 The TensorFlow Authors.**

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 클러스터링을 보존하는 희소성 Keras 예시

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/combine/sparse_clustering_example"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/model_optimization/guide/combine/sparse_clustering_example.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

## 개요

이것은 TensorFlow 모델 최적화 툴킷의 공동 작업 최적화 파이프라인의 일부인 **클러스터링을 보존하는 희소성** API의 사용을 보여주는 엔드 투 엔드 예시입니다.

### 다른 페이지

파이프라인 및 기타 가능한 기술에 대한 지침의 경우, [공동 협력 최적화 개요 페이지](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization)를 참조합니다.

### 내용

이 튜토리얼에서는 다음을 수행합니다.

1. MNIST 데이터세트를 위한 `tf.keras` 모델을 처음부터 훈련합니다.
2. 희소성으로 모델을 미세 조정하고 정확성을 확인하며 해당 모델이 성공적으로 잘라내기 되었는지 관찰합니다.
3. 잘라내기 된 모델에 가충치 클러스터링을 적용하고 희소성 손실을 관찰합니다.
4. 잘라내기 된 모델에 클러스터링을 유지하는 희소성을 적용하고 이전에 적용된 희소성이 유지되었는지 관찰합니다.
5. TFLite 모델을 생성하고 잘라내기 된 클러스터링 된 모델에서 정확성이 유지되었는지 확인합니다.
6. 여러 모델의 규모를 비교하여 희소성 적용의 압축 이점에 이어 클러스터링을 보존하는 희소성의 공동 협력 최적화 기술을 관찰합니다.

## 설치

이 Jupyter 노트북은 로컬 [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) 또는 [colab](https://colab.sandbox.google.com/)에서 실행할 수 있습니다. 종속성 설정에 대한 자세한 내용은 [설치 가이드](https://www.tensorflow.org/model_optimization/guide/install)를 참조하세요. 

In [ ]:
! pip install -q tensorflow-model-optimization

In [ ]:
import tensorflow as tf

import numpy as np
import tempfile
import zipfile
import os

## 잘라내기 되고 클러스터링 될 MNIST용 tf.keras 모델 훈련하기

In [ ]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3),
                         activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

### 기준 모델을 평가하고 나중에 사용할 수 있도록 저장하기

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

## 50%의 희소성으로 모델 잘라내기 및 미세 조정하기

`prune_low_magnitude()` API를 적용하여 사전 훈련된 전체 모델을 잘라내어 다음 단계에서 클러스터링 될 모델을 구축합니다. 대상 정확성을 유지하면서 최적의 압축률을 구축하는 API를 사용하는 최적의 방법은 [잘라내기 종합 가이드](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide)를 참조하세요.

### 모델 정의 및 희소성 API 적용하기

사전 훈련된 모델이 사용된 점을 참조하십시오.

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
  }

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model = prune_low_magnitude(model, **pruning_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

pruned_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

pruned_model.summary()

### 모델을 미세 조정하고, 희소성을 확인하고, 기준 대비 정확성 평가하기

3번의 epochs 동안 잘라내기로 모델을 미세 조정합니다.

In [ ]:
# Fine-tune model
pruned_model.fit(
  train_images,
  train_labels,
  epochs=3,
  validation_split=0.1,
  callbacks=callbacks)

도우미 함수를 정의하여 모델의 희소성을 계산하고 프린트합니다.

In [ ]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )

모델 커널이 올바르게 잘라내기 되었는지 확인합니다. 잘라내기 래퍼를 우선 분해해야 합니다. 또한 다음 단계에 사용될 모델의 깊은 복사를 생성해야 합니다.

In [ ]:
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

print_model_weights_sparsity(stripped_pruned_model)

stripped_pruned_model_copy = tf.keras.models.clone_model(stripped_pruned_model)
stripped_pruned_model_copy.set_weights(stripped_pruned_model.get_weights())

## 클러스터링을 보존하는 클러스터링 및 희소성을 적용하고 두 사례에서 모델 희소성에 대한 효과 확인하기

다음으로, 잘라내기 된 모델에 클러스터링을 보존하는 클러스터링 및 희소성을 모두 적용하고 잘라내기 된 모델의 후자가 희소성을 유지하는지 관찰합니다. 클러스터링 API를 적용하기 전 `tfmot.sparsity.keras.strip_pruning`로 잘라내기 된 모델의 잘라내기 래퍼를 분해했다는 점을 참고합니다.

In [ ]:
# Clustering
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(stripped_pruned_model, **clustering_params)

clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print('Train clustering model:')
clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)


stripped_pruned_model.save("stripped_pruned_model_clustered.h5")

# Sparsity preserving clustering
from tensorflow_model_optimization.python.core.clustering.keras.experimental import (
    cluster,
)

cluster_weights = cluster.cluster_weights

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS,
  'preserve_sparsity': True
}

sparsity_clustered_model = cluster_weights(stripped_pruned_model_copy, **clustering_params)

sparsity_clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print('Train sparsity preserving clustering model:')
sparsity_clustered_model.fit(train_images, train_labels,epochs=3, validation_split=0.1)

두 모델 모두에 대해 희소성을 확인합니다.

In [ ]:
print("Clustered Model sparsity:\n")
print_model_weights_sparsity(clustered_model)
print("\nSparsity preserved clustered Model sparsity:\n")
print_model_weights_sparsity(sparsity_clustered_model)

## 클러스터링으로부터 1.6배 더 작은 모델 만들기

도우미 함수를 정의하여 압축된 모델 파일을 얻습니다.

In [ ]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in kilobytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)/1000

In [ ]:
# Clustered model
clustered_model_file = 'clustered_model.h5'

# Save the model.
clustered_model.save(clustered_model_file)
    
#Sparsity Preserve Clustered model
sparsity_clustered_model_file = 'sparsity_clustered_model.h5'

# Save the model.
sparsity_clustered_model.save(sparsity_clustered_model_file)
    
print("Clustered Model size: ", get_gzipped_model_size(clustered_model_file), ' KB')
print("Sparsity preserved clustered Model size: ", get_gzipped_model_size(sparsity_clustered_model_file), ' KB')

## 가중치 클러스팅 및 훈련 후 양자화를 보존하는 희소성 조합에서 TFLite 모델 생성하기

클러스터링 래퍼를 분해하고 TFLite로 변환합니다.

In [ ]:
stripped_sparsity_clustered_model = tfmot.clustering.keras.strip_clustering(sparsity_clustered_model)

converter = tf.lite.TFLiteConverter.from_keras_model(stripped_sparsity_clustered_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
sparsity_clustered_quant_model = converter.convert()

_, pruned_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_and_clustered_tflite_file, 'wb') as f:
  f.write(sparsity_clustered_quant_model)

print("Sparsity preserved clustered Model size: ", get_gzipped_model_size(sparsity_clustered_model_file), ' KB')
print("Sparsity preserved clustered and quantized TFLite model size:",
       get_gzipped_model_size(pruned_and_clustered_tflite_file), ' KB')

## TF에서 TFLite로 정확성이 지속되는지 확인하기

In [ ]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print(f"Evaluated on {i} results so far.")
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

잘라내기 되고, 클러스터링 되고 양자화된 모델을 평가한 다음 TFLite 백엔드에서 TensorFlow의 정확성이 유지되는지 확인합니다.

In [ ]:
# Keras model evaluation
stripped_sparsity_clustered_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
_, sparsity_clustered_keras_accuracy = stripped_sparsity_clustered_model.evaluate(
    test_images, test_labels, verbose=0)

# TFLite model evaluation
interpreter = tf.lite.Interpreter(pruned_and_clustered_tflite_file)
interpreter.allocate_tensors()

sparsity_clustered_tflite_accuracy = eval_model(interpreter)

print('Pruned, clustered and quantized Keras model accuracy:', sparsity_clustered_keras_accuracy)
print('Pruned, clustered and quantized TFLite model accuracy:', sparsity_clustered_tflite_accuracy)

## 결론

이 튜토리얼에서, 모델을 생성하고 `prune_low_magnitude()` API를 사용하여 이를 잘라내고 클러스터링을 보존하는 희소성을 적용하여 가중치를 클러스터링 하면서 희소성을 유지하는 방법을 학습했습니다. 클러스터링 된 모델을 보존하는 희소성은 클러스터링 된 모델과 비교되어 전자의 경우 희소성이 보존되고 후자의 경우 손실된다는 점을 보여주었습니다. 다음으로, 클러스터링 된 잘라내기 된 모델은 TFLite로 변환되어 잘라내기 연결의 압축 이점과 클러스터링 모델 최적화 기술을 보존하는 희소성을 보여주었고, 마지막으로, TFLite 모델은 TFLite 백엔드에서 희소성을 유지하도록 평가되었습니다.